In [ ]:
import pandas as pd
import importlib
import q_learning
importlib.reload(q_learning)
from q_learning import run
import numpy as np

### Creation of Q_learning tables

In [ ]:
from itertools import product
is_training = [True]
is_slippery = [False, True]
n_episodes = [15000]
map_names = ['4x4', '8x8', '9x9', '10x10']
seed = [20] #20 works for 4 and 8, 16 and 32 are too big
learning_rate_as = [0.01, 0.1, 0.9] #0.1, 0.01, 0.5 works
discount_factor_gammas = [0.01, 0.5, 0.95] #0.1, 0.5, 0.95 works
epsilon = [1]
epsilon_decay_rates = [0.0001]
render = [False]
hyperparameter_grid = list(product(is_training, render, is_slippery, n_episodes, map_names, seed, learning_rate_as, discount_factor_gammas, epsilon, epsilon_decay_rates))

In [ ]:
results = []
num_hyper = len(hyperparameter_grid)
print(f"Running {num_hyper} hyperparameter combinations")

for hyperparameters in hyperparameter_grid:
    is_training, render, is_slippery, n_episodes, map_name, seed, learning_rate_a, discount_factor_gamma, epsilon, epsilon_decay_rate = hyperparameters
    rewards_per_episode = run(is_training=is_training, render=render, is_slippery=is_slippery, episodes=n_episodes, map_name=map_name, seed=seed, learning_rate_a=learning_rate_a, discount_factor_g=discount_factor_gamma, epsilon=epsilon, epsilon_decay_rate=epsilon_decay_rate)
    results.append({"is_training": is_training, "render": render, "is_slippery": is_slippery, "n_episodes": n_episodes, "map_name": map_name, "seed": seed, "learning_rate_a": learning_rate_a, "discount_factor_gamma": discount_factor_gamma, "epsilon": epsilon, "epsilon_decay_rate": epsilon_decay_rate, "rewards_per_episode": rewards_per_episode})
    print(f"{len(results)/num_hyper*100}% done")
    
df = pd.DataFrame(results)

In [ ]:
def calc_sum_rewards(rewards_per_episode):
    sum_rewards = np.zeros(rewards_per_episode.shape[0])
    for t in range(rewards_per_episode.shape[0]):
        sum_rewards[t] = np.sum(rewards_per_episode[max(0, t-100):(t+1)])
        
    return sum_rewards

In [ ]:
df['sum_rewards'] = df['rewards_per_episode'].apply(calc_sum_rewards)

In [ ]:
#save df to csv
import csv
df.to_csv('./results/frozen_lake_results.csv', index=False, quoting=csv.QUOTE_NONE)
df.to_pickle('./results/frozen_lake_results.pkl')